<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 603.9 MB 21 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [ ]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span, Token
from spacy import displacy

In [ ]:
markets = ['體育用品市場']
CHECKPOINT = 'zh_core_web_lg'

In [ ]:
nlp = spacy.load(CHECKPOINT)

nlp.tokenizer.initialize(pkuseg_model="mixed")
nlp.disable_pipe('ner')

sentence = '體育用品市場'
doc = nlp(sentence)
for token in doc:
    print(token.text,token.pos_)

體育 NOUN
用品 NOUN
市場 NOUN


In [ ]:
market_patterns = [[{"TEXT": market}] for market in markets]

def label_market(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    token = doc[start:end]
    next_token = doc[end:end+1]
    new_ents = []
    for ent in doc.ents:
      new_ents.append(ent)
    new_ent = Span(doc, start, end, label="FIN-MARKET")
    new_ents.append(new_ent)
    doc.ents = new_ents

nlp = spacy.load(CHECKPOINT)
user_dict = markets

nlp.tokenizer.initialize(pkuseg_model="mixed")
nlp.tokenizer.pkuseg_update_user_dict(user_dict)
nlp.disable_pipe('ner')

matcher = Matcher(nlp.vocab)
matcher.add('market',market_patterns,on_match=label_market)

sentence = '體育用品市場'
doc = nlp(sentence)
matches = matcher(doc)

displacy.render(doc, jupyter=True, style="ent")
